In [ ]:
import sys

sys.path.append("/home/jovyan/work/sem-covid/")
sys.path = list(set(sys.path))
import os

os.getcwd()
os.chdir('/home/jovyan/work/sem-covid/')

from sem_covid.services.data_registry import Dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from statistics import mean

In [ ]:
vectorizer = TfidfVectorizer(lowercase=True, stop_words='english')

In [ ]:
pwdb = Dataset.PWDB.fetch()[['background_info_description', 'content_of_measure_description']]

In [ ]:

get_weights_chunks(pwdb['content_of_measure_description'][0].split('.'))
